<a href="https://colab.research.google.com/github/labib899/CSE-837-ML/blob/main/hw5/logistic-regression-lab-exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚯 Lecture 11 Lab: Logistic regression and spam detection

<img src="https://github.com/joshuagrossman/mse125-labs-public/blob/main/hw5/img/spam-email.png?raw=1" alt= “spam-email” width="500" />

## ✅ Setup and data import
In this lab, we will work with a [classic dataset](https://archive.ics.uci.edu/dataset/94/spambase) of 4,601 emails classified as spam or not spam.

In [1]:
# Load in additional functions
library(tidyverse)
library(lubridate)

# Use three digits past the decimal point,
# and don't use scientific notation.
options(digits = 3, scipen = 999)

# Format plots with a white background and dark features.
theme_set(theme_bw())

# Increase the default text size of plots.
# If you are *not* working in Google Colab, we recommend commenting
# out this line of code.
theme_update(text = element_text(size = 20))

# Increase the default plot width and height.
# If you are *not* working in Google Colab, we recommend commenting
# out this line of code.
options(repr.plot.width=12, repr.plot.height=8)

# Read in the data
spam = read_csv('https://jdgrossman.com/assets/spam.csv')

# peek at 10 random rows
sample_n(spam, 10)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.1     ✔ stringr   1.6.0
✔ ggplot2   4.0.0     ✔ tibble    3.3.0
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.2.0     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Rows: 4601 Columns: 58
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
dbl (58): make, address, all, 3d, our, over, remove, internet, order, mail, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


make,address,all,3d,our,over,remove,internet,order,mail,⋯,char_semicolon,char_left_paren,char_left_bracket,char_exclamation,char_dollar,char_pound,capital_run_length_average,capital_run_length_longest,capital_run_length_total,is_spam
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0.00,0.00,0.42,0,0.00,0.00,0.00,0,0.00,0,⋯,0.000,0.078,0.039,0.039,0.000,0.000,1.29,6,106,0
0.12,0.00,0.12,0,0.00,0.00,0.00,0,1.11,0,⋯,0.119,0.170,0.034,0.000,0.034,0.000,3.24,32,505,0
0.19,0.00,0.38,0,0.00,0.19,0.00,0,0.00,0,⋯,0.010,0.010,0.000,0.000,0.000,0.003,2.38,21,15841,1
0.00,0.00,0.00,0,0.45,0.45,0.45,0,0.00,0,⋯,0.000,0.000,0.000,0.072,0.000,1.083,7.86,45,440,1
0.00,0.00,0.00,0,0.00,0.00,0.00,0,0.00,0,⋯,0.000,0.746,0.000,0.000,0.000,0.000,1.69,4,27,0
0.00,0.68,0.00,0,0.00,0.00,0.00,0,0.00,0,⋯,0.000,0.216,0.108,0.216,0.000,0.108,2.75,25,157,0
0.00,14.28,0.00,0,0.00,0.00,0.00,0,0.00,0,⋯,0.000,0.000,0.000,0.000,0.000,0.000,1.80,5,9,0
0.00,0.00,0.00,0,0.00,0.00,0.00,0,0.00,0,⋯,0.000,0.208,0.000,0.417,0.208,0.000,3.81,16,61,1
0.00,0.00,0.00,0,0.00,0.00,0.00,0,0.00,0,⋯,0.000,0.258,0.000,0.000,0.000,0.000,3.74,53,101,0


## ♨️ Warm up

How many emails are in the database?

What fraction of the emails in the database are spam?

Which email contains the highest percentage of words matching "money"? What percentage of words in that email match "money"?

In [8]:
nrow(spam)
mean(spam$is_spam)
i_max <- which.max(spam$money)
spam[i_max, ]
spam$money[i_max]

[1] 4601

[1] 0.394

make,address,all,3d,our,over,remove,internet,order,mail,⋯,char_semicolon,char_left_paren,char_left_bracket,char_exclamation,char_dollar,char_pound,capital_run_length_average,capital_run_length_longest,capital_run_length_total,is_spam
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,1,1,2,1


[1] 12.5

## 🎲 Linear probability models (LPMs)

Fit a linear regression model to the spam data with the `lm` function.

Use the following covariates to predict the likelihood that an email is spam:
- `char_dollar`
- `credit`
- `money`
- `re`

How would you interpret the model coefficients for the intercept and for `char_dollar`?

- Note: `char_dollar` represents the percentage of characters in the email that match `$`.

In [9]:
model <- lm(is_spam ~ char_dollar + credit + money + re, data = spam)
summary(model)


Call:
lm(formula = is_spam ~ char_dollar + credit + money + re, data = spam)

Residuals:
   Min     1Q Median     3Q    Max 
-2.849 -0.335 -0.303  0.514  0.963 

Coefficients:
            Estimate Std. Error t value            Pr(>|t|)    
(Intercept)  0.33459    0.00732   45.70 <0.0000000000000002 ***
char_dollar  0.58551    0.02675   21.89 <0.0000000000000002 ***
credit       0.15752    0.01285   12.26 <0.0000000000000002 ***
money        0.18794    0.01487   12.64 <0.0000000000000002 ***
re          -0.05355    0.00648   -8.27 <0.0000000000000002 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.443 on 4596 degrees of freedom
Multiple R-squared:  0.179,	Adjusted R-squared:  0.178 
F-statistic:  250 on 4 and 4596 DF,  p-value: <0.0000000000000002


Using your linear probability model and the `predict` function, predict the in-sample probability that each email is spam.

What is the smallest predicted probability? The largest? Do you notice any issues with these predictions?

In [10]:
pred <- predict(model)
summary(pred)
min(pred)
max(pred)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 -0.813   0.335   0.335   0.394   0.395   3.849 

[1] -0.813

[1] 3.85

## 🎰 Odds functions

Write two functions:
- A function to convert probabilities to odds.
- A function to convert odds to probabilities

Test your functions by making sure that 2:1 odds returns a 2/3 probability, and vice versa.

Finally, suppose my probability of winning is 60%. If I double my odds of winning, what is my new probability of winning?

In [14]:
prob_to_odds <- function(p) {
  p / (1 - p)
}

odds_to_prob <- function(o) {
  o / (1 + o)
}

odds_to_prob(2)
prob_to_odds(2/3)
odds_to_prob(prob_to_odds(0.6)*2)


[1] 0.667

[1] 2

[1] 0.75

## 🪙 Fitting a logistic regression model

We can fit a logistic regression model with the same covariates as above with the following code:

In [15]:
model = glm(is_spam ~ 1 + char_dollar + credit + money + re, family='binomial', data=spam)

summary(model)

Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”



Call:
glm(formula = is_spam ~ 1 + char_dollar + credit + money + re, 
    family = "binomial", data = spam)

Coefficients:
            Estimate Std. Error z value             Pr(>|z|)    
(Intercept)  -1.0666     0.0432  -24.68 < 0.0000000000000002 ***
char_dollar  11.8176     0.6045   19.55 < 0.0000000000000002 ***
credit        2.3119     0.3430    6.74   0.0000000000157692 ***
money         1.9933     0.2485    8.02   0.0000000000000010 ***
re           -0.7755     0.0994   -7.81   0.0000000000000059 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 6170.2  on 4600  degrees of freedom
Residual deviance: 4427.8  on 4596  degrees of freedom
AIC: 4438

Number of Fisher Scoring iterations: 7


Interpret the intercept and `money` coefficients for the logistic regression model three different ways:
1. On the log odds scale
2. On the odds scale (by exponentiating the coefficients)
3. On the probability scale (using either the odds functions you wrote, or the divide by 4 trick).

Tip: Use the `coef` function to extract coefficients from the model.

In [22]:
coef(model)
exp(coef(model))
odds_to_prob(exp(coef(model)))

(Intercept) char_dollar      credit       money          re 
     -1.067      11.818       2.312       1.993      -0.776

(Intercept) char_dollar      credit       money          re 
      0.344  135613.914      10.094       7.340       0.460

(Intercept) char_dollar      credit       money          re 
      0.256       1.000       0.910       0.880       0.315